In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import warnings
import gc
import time

# --- Core Scikit-learn ---
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# --- NEW: MLP Model ---
from sklearn.neural_network import MLPClassifier 

# --- GBDT Models ---
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool

# --- Blending ---
from scipy.optimize import minimize
from scipy.stats import rankdata

warnings.simplefilter('ignore')
print("="*80)
print("🚀 ULTIMATE 5-MODEL STACK (REVERSE-ENGINEERED) 🚀")
print("="*80)

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================
SEED_LIST = [42, 123]  # Multi-seed for stability
N_SPLITS = 5           # 5-Folds for speed
TARGET = 'loan_paid_back'

In [ ]:
# ============================================================================
# LOAD DATA
# ============================================================================
print("\n[LOADING DATA]")
train = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')
orig = pd.read_csv('/kaggle/input/loan-prediction-dataset-2025/loan_dataset_20000.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')

# ⚠️ CRITICAL: Store original lengths BEFORE any modifications
TRAIN_LEN = len(train)
TEST_LEN = len(test)

print(f"Train: {train.shape}, Test: {test.shape}, Orig: {orig.shape}")

In [ ]:
# ============================================================================
# FEATURE ENGINEERING (FROM PRO MODEL)
# ============================================================================
print("\n[STARTING FEATURE ENGINEERING...]")
BASE = [col for col in train.columns if col not in ['id', TARGET]]
CATS = ['gender', 'marital_status', 'education_level', 'employment_status', 
        'loan_purpose', 'grade_subgrade']
NUMS = [col for col in BASE if col not in CATS]

# ──────────────────────────────────────────────────────────────────────────
# 1) ADVANCED ORIG STATISTICS
# ──────────────────────────────────────────────────────────────────────────
print("\n[1/4] Advanced Orig Statistics...")
ORIG_STATS = []
for col in BASE:
    for stat in ['mean', 'std', 'min', 'max', 'median']:
        stat_map = orig.groupby(col)[TARGET].agg(stat)
        stat_map.name = f"orig_{stat}_{col}"
        train = train.merge(stat_map, on=col, how='left')
        test = test.merge(stat_map, on=col, how='left')
        ORIG_STATS.append(f"orig_{stat}_{col}")
    
    count_map = orig.groupby(col).size().reset_index(name=f"orig_count_{col}")
    train = train.merge(count_map, on=col, how='left')
    test = test.merge(count_map, on=col, how='left')
    ORIG_STATS.append(f"orig_count_{col}")
print(f"✓ Created {len(ORIG_STATS)} advanced orig features")

# ──────────────────────────────────────────────────────────────────────────
# 2) BIGRAM & 3-WAY INTERACTIONS
# ──────────────────────────────────────────────────────────────────────────
print("\n[2/4] Interaction Features...")

# Add placeholder target for concatenation
test[TARGET] = -1
# Concat ALL 3 datasets for robust feature creation
combine = pd.concat([train, test, orig], axis=0, ignore_index=True)

INTER = []
TE_BASE = [col for col in BASE if col not in ['annual_income', 'loan_amount']]
for col1, col2 in combinations(TE_BASE, 2):
    new_col = f'{col1}_{col2}'
    INTER.append(new_col)
    combine[new_col] = combine[col1].astype(str) + "_" + combine[col2].astype(str)

INTER_3WAY = []
triplets = [
    ('grade_subgrade', 'employment_status', 'loan_purpose'),
    ('grade_subgrade', 'debt_to_income_ratio', 'credit_score'),
    ('loan_purpose', 'education_level', 'marital_status')]
for col1, col2, col3 in triplets:
    new_col = f'{col1}_{col2}_{col3}'
    INTER_3WAY.append(new_col)
    combine[new_col] = (combine[col1].astype(str) + "_" + 
                       combine[col2].astype(str) + "_" + 
                       combine[col3].astype(str))
print(f"✓ Created {len(INTER)} bigrams + {len(INTER_3WAY)} 3-way interactions")

# ──────────────────────────────────────────────────────────────────────────
# 3) QUANTILE FEATURES
# ──────────────────────────────────────────────────────────────────────────
print("\n[3/4] Quantile Features...")
QFEAT = []
for col in ['annual_income', 'loan_amount', 'credit_score']:
    # Create quantiles from the training data only to prevent leakage
    quantiles = np.percentile(combine.iloc[:TRAIN_LEN][col].dropna(), np.arange(0, 101, 5))
    qcol = f'{col}_quantile'
    combine[qcol] = np.digitize(combine[col], quantiles)
    QFEAT.append(qcol)
print(f"✓ Created {len(QFEAT)} quantile features")

# ──────────────────────────────────────────────────────────────────────────
# 4) ROUNDING FEATURES
# ──────────────────────────────────────────────────────────────────────────
print("\n[4/4] Rounding Features...")
ROUND = []
rounding_levels = {'1s': 0, '10s': -1, '100s': -2}
for col in ['annual_income', 'loan_amount']:
    for suffix, level in rounding_levels.items():
        new_col = f"{col}_ROUND_{suffix}"
        ROUND.append(new_col)
        combine[new_col] = combine[col].round(level).astype(int)
print(f"✓ Created {len(ROUND)} rounding features")
print(f"\n✅ Feature Engineering Complete")

In [ ]:
# ============================================================================
# ⚠️ CRITICAL FIX: PROPER DATA SPLITTING ⚠️
# ============================================================================
print('\n' + '='*80)
print('SPLITTING DATA (FIXED INDEXING)')
print('='*80)

# Split using EXACT row counts stored at the beginning
train = combine.iloc[:TRAIN_LEN].copy()
test = combine.iloc[TRAIN_LEN:TRAIN_LEN + TEST_LEN].copy()
orig = combine.iloc[TRAIN_LEN + TEST_LEN:].copy() # Keep orig for GBDT augmentation

# Verify splits
print(f"✓ Train: {train.shape} (expected {TRAIN_LEN} rows)")
print(f"✓ Test: {test.shape} (expected {TEST_LEN} rows)")
print(f"✓ Orig: {orig.shape} (expected {orig.shape[0]} rows)")

# Remove placeholder target from test
test = test.drop(columns=[TARGET])
del combine
gc.collect()
print(f'\n✅ Data split verified successfully!')

# ============================================================================
# FINAL FEATURE SETS
# ============================================================================
# All new string/quantile features to be Target Encoded
COLS_TO_TE = INTER + INTER_3WAY + QFEAT + ROUND

# Features for GBDTs (XGB, LGBM, CAT)
FEATURES_GBDT_BASE = BASE + ORIG_STATS + QFEAT + ROUND

# Features for Linear/MLP models
CATS_LR_MLP = ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']
NUMS_LR_MLP = NUMS + ORIG_STATS # Use original nums + all new numeric orig stats
FEATURES_LR_MLP_BASE = NUMS_LR_MLP + CATS_LR_MLP

print(f"\nTotal features to Target Encode: {len(COLS_TO_TE)}")

In [ ]:
# ============================================================================
# TARGET ENCODER CLASS (CPU-SAFE, ROBUST)
# ============================================================================
class TargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_encode, aggs=['mean'], cv=5, smooth=1.0, drop_original=False):
        self.cols_to_encode = cols_to_encode
        self.aggs = aggs
        self.cv = cv
        self.smooth = smooth
        self.drop_original = drop_original
        self.mappings_ = {}
        self.global_stats_ = {}

    def fit(self, X, y):
        temp_df = X.copy()
        temp_df['target'] = y
        for agg_func in self.aggs:
            self.global_stats_[agg_func] = y.agg(agg_func)
        for col in self.cols_to_encode:
            self.mappings_[col] = {}
            for agg_func in self.aggs:
                mapping = temp_df.groupby(col)['target'].agg(agg_func)
                self.mappings_[col][agg_func] = mapping
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for col in self.cols_to_encode:
            for agg_func in self.aggs:
                new_col_name = f'TE_{col}_{agg_func}'
                map_series = self.mappings_[col][agg_func]
                X_transformed[new_col_name] = X[col].map(map_series)
                X_transformed[new_col_name].fillna(self.global_stats_[agg_func], inplace=True)
        if self.drop_original:
            X_transformed.drop(columns=self.cols_to_encode, inplace=True)
        return X_transformed

    def fit_transform(self, X, y):
        self.fit(X, y)
        encoded_features = pd.DataFrame(index=X.index)
        kf = KFold(n_splits=self.cv, shuffle=True, random_state=42)
        
        for train_idx, val_idx in kf.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val = X.iloc[val_idx]
            temp_df_train = X_train.copy()
            temp_df_train['target'] = y_train
            
            for col in self.cols_to_encode:
                for agg_func in self.aggs:
                    new_col_name = f'TE_{col}_{agg_func}'
                    fold_global_stat = y_train.agg(agg_func)
                    mapping = temp_df_train.groupby(col)['target'].agg(agg_func)
                    
                    if agg_func == 'mean':
                        counts = temp_df_train.groupby(col)['target'].count()
                        m = self.smooth
                        smoothed_mapping = (counts * mapping + m * fold_global_stat) / (counts + m)
                        encoded_values = X_val[col].map(smoothed_mapping)
                    else:
                        encoded_values = X_val[col].map(mapping)
                    
                    encoded_features.loc[X_val.index, new_col_name] = encoded_values.fillna(fold_global_stat)
        
        X_transformed = X.copy()
        for col in encoded_features.columns:
            if encoded_features[col].isnull().any():
                encoded_features[col].fillna(self.global_stats_[agg_func], inplace=True)
            X_transformed[col] = encoded_features[col]
        
        if self.drop_original:
            X_transformed.drop(columns=self.cols_to_encode, inplace=True)
        
        return X_transformed

print("\n✓ TargetEncoder class defined")

In [ ]:
# ============================================================================
# CRITICAL FIX: DTYPE CONVERSION FUNCTION
# ============================================================================
def fix_dtypes_for_lgb(df, categorical_cols):
    """Fix dtypes for LightGBM compatibility."""
    df_fixed = df.copy()
    for col in df_fixed.columns:
        if col in categorical_cols:
            df_fixed[col] = df_fixed[col].astype('category')
        elif df_fixed[col].dtype == 'object':
            # Factorize any remaining object columns
            df_fixed[col], _ = pd.factorize(df_fixed[col])
    return df_fixed

print("✓ LGBM Dtype Fix function defined")

In [ ]:
# ============================================================================
# L1 TRAINING FUNCTION (ALL 5 MODELS)
# ============================================================================
def train_models(seed):
    """Train all 5 models with proper FE and dtype handling for a given seed"""
    
    print(f"\n{'─'*80}")
    print(f"SEED: {seed} - STARTING TRAINING")
    print(f"{'─'*80}")
    
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=seed)
    
    # Init OOF/Test arrays for this seed
    oof_lgb = np.zeros(TRAIN_LEN)
    test_lgb = np.zeros(TEST_LEN)
    oof_cat = np.zeros(TRAIN_LEN)
    test_cat = np.zeros(TEST_LEN)
    oof_xgb = np.zeros(TRAIN_LEN)
    test_xgb = np.zeros(TEST_LEN)
    oof_mlp = np.zeros(TRAIN_LEN)
    test_mlp = np.zeros(TEST_LEN)
    oof_lr = np.zeros(TRAIN_LEN)
    test_lr = np.zeros(TEST_LEN)

    # We must loop fold by fold because the FE is different for GBDTs vs Linear/MLP
    for fold, (train_idx, val_idx) in enumerate(skf.split(train, train[TARGET]), 1):
        print(f"\n--- Fold {fold}/{N_SPLITS} ---")
        
        # -------------------------------------------------
        # 1. Create Data for this Fold
        # -------------------------------------------------
        X_train_fold = train.iloc[train_idx].copy()
        y_train_fold = train.iloc[train_idx][TARGET].copy() # Base train target
        X_valid_fold = train.iloc[val_idx].copy()
        y_valid_fold = train.iloc[val_idx][TARGET].copy() # Base validation target
        X_test_fold = test.copy()

        # -------------------------------------------------
        # 2. Create GBDT data (LGBM, CAT, XGB)
        # -------------------------------------------------
        
        # Add orig data for GBDT training
        Xy_train_fold_gbdt = pd.concat([X_train_fold, orig], axis=0, ignore_index=True)
        y_train_fold_gbdt = Xy_train_fold_gbdt[TARGET] # This is the augmented training target

        # Apply Target Encoding
        te_gbdt = TargetEncoder(cols_to_encode=COLS_TO_TE, cv=5, smooth=1.0, aggs=['mean'], drop_original=True)
        X_train_gbdt = te_gbdt.fit_transform(Xy_train_fold_gbdt, y_train_fold_gbdt)
        X_valid_gbdt = te_gbdt.transform(X_valid_fold.copy())
        X_test_gbdt = te_gbdt.transform(X_test_fold.copy())

        # Define GBDT features
        FEATURES_GBDT_FINAL = [col for col in X_train_gbdt.columns if col not in ['id', TARGET]]
        
        # -------------------------------------------------
        # 3. Create LR/MLP data (Linear, NN)
        # -------------------------------------------------
        
        # Apply Target Encoding (this "feeds" the models)
        te_linear = TargetEncoder(cols_to_encode=COLS_TO_TE, cv=5, smooth=1.0, aggs=['mean'], drop_original=True)
        X_train_linear = te_linear.fit_transform(X_train_fold.copy(), y_train_fold) # Use base train fold for fit
        X_valid_linear = te_linear.transform(X_valid_fold.copy())
        X_test_linear = te_linear.transform(X_test_fold.copy())
        
        # Get list of new TE features
        TE_FEATURES = [col for col in X_train_linear.columns if 'TE_' in col]
        
        # Define final LR/MLP feature list
        NUMS_LR_MLP_FINAL = NUMS_LR_MLP + TE_FEATURES
        FEATURES_LR_MLP_FINAL = NUMS_LR_MLP_FINAL + CATS_LR_MLP

        X_train_linear = X_train_linear[FEATURES_LR_MLP_FINAL].copy()
        X_valid_linear = X_valid_linear[FEATURES_LR_MLP_FINAL].copy()
        X_test_linear = X_test_linear[FEATURES_LR_MLP_FINAL].copy()
        
        # Fill NaNs
        X_train_linear[NUMS_LR_MLP_FINAL] = X_train_linear[NUMS_LR_MLP_FINAL].fillna(0)
        X_valid_linear[NUMS_LR_MLP_FINAL] = X_valid_linear[NUMS_LR_MLP_FINAL].fillna(0)
        X_test_linear[NUMS_LR_MLP_FINAL] = X_test_linear[NUMS_LR_MLP_FINAL].fillna(0)
        
        # One-Hot Encode
        X_train_linear = pd.get_dummies(X_train_linear, columns=CATS_LR_MLP, drop_first=True)
        X_valid_linear = pd.get_dummies(X_valid_linear, columns=CATS_LR_MLP, drop_first=True)
        X_test_linear = pd.get_dummies(X_test_linear, columns=CATS_LR_MLP, drop_first=True)
        
        # Align
        X_train_linear, X_valid_linear = X_train_linear.align(X_valid_linear, join='left', axis=1, fill_value=0)
        X_train_linear, X_test_linear = X_train_linear.align(X_test_linear, join='left', axis=1, fill_value=0)
        
        # -------------------------------------------------
        # 4. Train Models
        # -------------------------------------------------
        
        # [1/5] LightGBM
        print("  [1/5] Training LGBM...")
        params_lgb = {
            'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt',
            'learning_rate': 0.008, 'num_leaves': 32, 'max_depth': 6,
            'min_child_samples': 20, 'subsample': 0.75, 'colsample_bytree': 0.35,
            'reg_alpha': 1.2, 'reg_lambda': 5.5, 'random_state': seed,
            'device': 'gpu', 'verbose': -1, 'n_estimators': 12000
        }
        lgb_cat_cols = [c for c in CATS + QFEAT if c in X_train_gbdt.columns]
        X_train_lgb = fix_dtypes_for_lgb(X_train_gbdt[FEATURES_GBDT_FINAL], lgb_cat_cols)
        X_valid_lgb = fix_dtypes_for_lgb(X_valid_gbdt[FEATURES_GBDT_FINAL], lgb_cat_cols)
        X_test_lgb = fix_dtypes_for_lgb(X_test_gbdt[FEATURES_GBDT_FINAL], lgb_cat_cols)

        model_lgb = LGBMClassifier(**params_lgb)
        model_lgb.fit(X_train_lgb, y_train_fold_gbdt, 
                      eval_set=[(X_valid_lgb, y_valid_fold)],
                      callbacks=[lgb.early_stopping(200, verbose=False)],
                      categorical_feature=[c for c in lgb_cat_cols if c in X_train_lgb.columns])
        
        oof_lgb[val_idx] = model_lgb.predict_proba(X_valid_lgb)[:, 1]
        test_lgb += model_lgb.predict_proba(X_test_lgb)[:, 1] / N_SPLITS
        del X_train_lgb, X_valid_lgb, X_test_lgb, model_lgb
        
        # [2/5] CatBoost
        print("  [2/5] Training CatBoost...")
        params_cat = {
            'iterations': 2500, 'learning_rate': 0.02, 'depth': 6, 'l2_leaf_reg': 3.5,
            'random_strength': 2.0, 'bagging_temperature': 0.5, 'task_type': 'GPU',
            'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': seed,
            'early_stopping_rounds': 150, 'verbose': False
        }
        cat_cols_for_catboost = [c for c in CATS + QFEAT if c in X_train_gbdt.columns]
        cat_indices = [X_train_gbdt[FEATURES_GBDT_FINAL].columns.get_loc(c) for c in cat_cols_for_catboost]
        
        train_pool = Pool(X_train_gbdt[FEATURES_GBDT_FINAL], y_train_fold_gbdt, cat_features=cat_indices)
        val_pool = Pool(X_valid_gbdt[FEATURES_GBDT_FINAL], y_valid_fold, cat_features=cat_indices)
        test_pool = Pool(X_test_gbdt[FEATURES_GBDT_FINAL], cat_features=cat_indices)
        
        model_cat = CatBoostClassifier(**params_cat)
        model_cat.fit(train_pool, eval_set=val_pool)
        
        oof_cat[val_idx] = model_cat.predict_proba(val_pool)[:, 1]
        test_cat += model_cat.predict_proba(test_pool)[:, 1] / N_SPLITS
        del train_pool, val_pool, test_pool, model_cat
        
        # [3/5] XGBoost
        print("  [3/5] Training XGBoost...")
        params_xgb = {
            'objective': 'binary:logistic', 'eval_metric': 'auc', 'max_depth': 6,
            'min_child_weight': 5, 'colsample_bytree': 0.35, 'colsample_bylevel': 0.65,
            'subsample': 0.70, 'reg_alpha': 1.2, 'reg_lambda': 4.5, 'gamma': 0.4,
            'learning_rate': 0.008, 'n_estimators': 15000, 'early_stopping_rounds': 250,
            'random_state': seed, 'n_jobs': -1, 'enable_categorical': True,
            'device': 'cuda', 'tree_method': 'hist'
        }
        xgb_cat_cols = [c for c in CATS + QFEAT if c in X_train_gbdt.columns]
        
        X_train_xgb = X_train_gbdt[FEATURES_GBDT_FINAL].copy()
        X_valid_xgb = X_valid_gbdt[FEATURES_GBDT_FINAL].copy()
        X_test_xgb = X_test_gbdt[FEATURES_GBDT_FINAL].copy()
        
        for cat_col in xgb_cat_cols:
            X_train_xgb[cat_col] = X_train_xgb[cat_col].astype('category')
            X_valid_xgb[cat_col] = X_valid_xgb[cat_col].astype('category')
            X_test_xgb[cat_col] = X_test_xgb[cat_col].astype('category')
        
        model_xgb = XGBClassifier(**params_xgb)
        model_xgb.fit(X_train_xgb, y_train_fold_gbdt, eval_set=[(X_valid_xgb, y_valid_fold)], verbose=False)
        
        oof_xgb[val_idx] = model_xgb.predict_proba(X_valid_xgb)[:, 1]
        test_xgb += model_xgb.predict_proba(X_test_xgb)[:, 1] / N_SPLITS
        del X_train_xgb, X_valid_xgb, X_test_xgb, model_xgb
        
        # [4/5] MLPClassifier
        print("  [4/5] Training MLP...")
        mlp_params = {
            'hidden_layer_sizes': (100, 50), 'activation': 'relu', 'solver': 'adam',
            'batch_size': 256, 'learning_rate': 'adaptive', 'max_iter': 1000,
            'early_stopping': True, 'n_iter_no_change': 20,
            'validation_fraction': 0.1, 'random_state': seed
        }
        pipe_mlp = Pipeline([
            ('scaler', StandardScaler()),
            ('model', MLPClassifier(**mlp_params))
        ])
        
        # --- FIX: Use y_train_fold (the base fold target) ---
        pipe_mlp.fit(X_train_linear, y_train_fold)
        
        oof_mlp[val_idx] = pipe_mlp.predict_proba(X_valid_linear)[:, 1]
        test_mlp += pipe_mlp.predict_proba(X_test_linear)[:, 1] / N_SPLITS
        del pipe_mlp
        
        # [5/5] Logistic Regression
        print("  [5/5] Training LR...")
        pipe_lr = Pipeline([
            ('scaler', StandardScaler()),
            ('model', LogisticRegression(C=0.1, max_iter=1000, random_state=seed, solver='liblinear'))
        ])
        
        # --- FIX: Use y_train_fold (the base fold target) ---
        pipe_lr.fit(X_train_linear, y_train_fold)
        
        oof_lr[val_idx] = pipe_lr.predict_proba(X_valid_linear)[:, 1]
        test_lr += pipe_lr.predict_proba(X_test_linear)[:, 1] / N_SPLITS
        del pipe_lr
        
        # Cleanup
        print(f"  Fold {fold} complete.")
        del X_train_gbdt, y_train_fold_gbdt, X_valid_gbdt, X_test_gbdt
        del X_train_linear, y_valid_fold, X_valid_linear, X_test_linear
        del X_train_fold, y_train_fold, X_valid_fold, X_test_fold
        gc.collect()

    # Print fold scores
    print(f"\nSEED {seed} CV Scores:")
    print(f"  LGBM: {roc_auc_score(train[TARGET], oof_lgb):.5f}")
    print(f"  CAT:  {roc_auc_score(train[TARGET], oof_cat):.5f}")
    print(f"  XGB:  {roc_auc_score(train[TARGET], oof_xgb):.5f}")
    print(f"  MLP:  {roc_auc_score(train[TARGET], oof_mlp):.5f}")
    print(f"  LR:   {roc_auc_score(train[TARGET], oof_lr):.5f}")
    
    return oof_lgb, test_lgb, oof_cat, test_cat, oof_xgb, test_xgb, oof_mlp, test_mlp, oof_lr, test_lr

In [ ]:
# ============================================================================
# RUN TRAINING FOR ALL SEEDS
# ============================================================================
print("\n" + "="*80)
print("TRAINING MULTI-SEED 5-MODEL ENSEMBLE")
print("="*80)
start_time = time.time()

oof_lgbs, test_lgbs = [], []
oof_cats, test_cats = [], []
oof_xgbs, test_xgbs = [], []
oof_mlps, test_mlps = [], []
oof_lrs, test_lrs = [], []

for seed in SEED_LIST:
    out_lgb, tst_lgb, out_cat, tst_cat, out_xgb, tst_xgb, out_mlp, tst_mlp, out_lr, tst_lr = train_models(seed)
    
    oof_lgbs.append(out_lgb)
    test_lgbs.append(tst_lgb)
    oof_cats.append(out_cat)
    test_cats.append(tst_cat)
    oof_xgbs.append(out_xgb)
    test_xgbs.append(tst_xgb)
    oof_mlps.append(out_mlp)
    test_mlps.append(tst_mlp)
    oof_lrs.append(out_lr)
    test_lrs.append(tst_lr)

In [ ]:
# ============================================================================
# AVERAGE ACROSS SEEDS
# ============================================================================
print("\n" + "="*80)
print("AVERAGING PREDICTIONS ACROSS SEEDS")
print("="*80)

oof_lgb_avg = np.mean(oof_lgbs, axis=0)
test_lgb_avg = np.mean(test_lgbs, axis=0)
oof_cat_avg = np.mean(oof_cats, axis=0)
test_cat_avg = np.mean(test_cats, axis=0)
oof_xgb_avg = np.mean(oof_xgbs, axis=0)
test_xgb_avg = np.mean(test_xgbs, axis=0)
oof_mlp_avg = np.mean(oof_mlps, axis=0)
test_mlp_avg = np.mean(test_mlps, axis=0)
oof_lr_avg = np.mean(oof_lrs, axis=0)
test_lr_avg = np.mean(test_lrs, axis=0)

cv_lgb_final = roc_auc_score(train[TARGET], oof_lgb_avg)
cv_cat_final = roc_auc_score(train[TARGET], oof_cat_avg)
cv_xgb_final = roc_auc_score(train[TARGET], oof_xgb_avg)
cv_mlp_final = roc_auc_score(train[TARGET], oof_mlp_avg)
cv_lr_final = roc_auc_score(train[TARGET], oof_lr_avg)

print(f"LGBM Multi-Seed CV: {cv_lgb_final:.5f}")
print(f"CatBoost Multi-Seed CV: {cv_cat_final:.5f}")
print(f"XGBoost Multi-Seed CV:  {cv_xgb_final:.5f}")
print(f"MLP Multi-Seed CV:      {cv_mlp_final:.5f}")
print(f"LR Multi-Seed CV:       {cv_lr_final:.5f}")

# This is our L2 training data
all_oof_l1 = [oof_lgb_avg, oof_cat_avg, oof_xgb_avg, oof_mlp_avg, oof_lr_avg]
all_test_l1 = [test_lgb_avg, test_cat_avg, test_xgb_avg, test_mlp_avg, test_lr_avg]

In [ ]:
# ============================================================================
# LAYER 2: RANKING & ADVANCED STACKING
# ============================================================================
print('\n' + '='*80)
print('LAYER 2: RANKING & ADVANCED STACKING (5 MODELS)')
print('='*80)

meta_train = np.column_stack(all_oof_l1)
meta_test = np.column_stack(all_test_l1)

print("Converting OOF and Test predictions to ranks...")
meta_train_ranked = np.zeros_like(meta_train)
meta_test_ranked = np.zeros_like(meta_test)

for i in range(meta_train.shape[1]):
    meta_train_ranked[:, i] = rankdata(meta_train[:, i])
    meta_test_ranked[:, i] = rankdata(meta_test[:, i])

# --- Method 1: L2 LGBM Stacker (on Ranks) ---
print('\nTraining L2 LGBM Stacker...')
l2_lgbm_params = {
    'objective': 'binary', 'metric': 'auc', 'n_estimators': 1000,
    'learning_rate': 0.02, 'num_leaves': 7, 'max_depth': 3,
    'subsample': 0.8, 'colsample_bytree': 0.7, 'random_state': SEED_LIST[0],
    'n_jobs': -1, 'verbose': -1, 'device': 'gpu'
}

# We can use a simple KFold for the stacker
kf_meta = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED_LIST[0]+1)
oof_preds_l2_lgbm = np.zeros(TRAIN_LEN)
test_preds_l2_lgbm = np.zeros(TEST_LEN)

for fold, (train_idx, val_idx) in enumerate(kf_meta.split(meta_train_ranked, train[TARGET])):
    l2_model = LGBMClassifier(**l2_lgbm_params)
    l2_model.fit(meta_train_ranked[train_idx], train[TARGET].iloc[train_idx],
                 eval_set=[(meta_train_ranked[val_idx], train[TARGET].iloc[val_idx])],
                 callbacks=[lgb.early_stopping(100, verbose=False)])
    
    oof_preds_l2_lgbm[val_idx] = l2_model.predict_proba(meta_train_ranked[val_idx])[:, 1]
    test_preds_l2_lgbm += l2_model.predict_proba(meta_test_ranked)[:, 1] / N_SPLITS

cv_l2_lgbm = roc_auc_score(train[TARGET], oof_preds_l2_lgbm)
print(f'L2 LGBM Stacker Final CV: {cv_l2_lgbm:.5f}')

# --- Method 2: Optimized Weighted-Average (on Ranks) ---
print('\nOptimizing blend weights (on Ranks)...')

def get_auc_ranked(weights):
    w = weights / np.sum(weights)
    # Order: [LGBM, CAT, XGB, MLP, LR]
    weighted_oof_ranked = (w[0] * meta_train_ranked[:, 0] + # LGBM
                           w[1] * meta_train_ranked[:, 1] + # CAT
                           w[2] * meta_train_ranked[:, 2] + # XGB
                           w[3] * meta_train_ranked[:, 3] + # MLP
                           w[4] * meta_train_ranked[:, 4])  # LR
    return -roc_auc_score(train[TARGET], weighted_oof_ranked)

initial_weights = [0.2, 0.2, 0.2, 0.2, 0.2]
bounds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]
opt_result = minimize(get_auc_ranked, initial_weights, method='Nelder-Mead', bounds=bounds)

optimized_weights_ranked = opt_result.x / np.sum(opt_result.x)
cv_optimized_ranked = -opt_result.fun 

print(f"Optimized Weights (Ranked): LGBM={optimized_weights_ranked[0]:.4f}, CAT={optimized_weights_ranked[1]:.4f}, XGB={optimized_weights_ranked[2]:.4f}, MLP={optimized_weights_ranked[3]:.4f}, LR={optimized_weights_ranked[4]:.4f}")
print(f"Optimized Rank Blend CV: {cv_optimized_ranked:.5f}")

In [ ]:
# ============================================================================
# FINAL ENSEMBLE & SAVE
# ============================================================================
print('\n' + '='*80)
print('FINAL ENSEMBLE (ON RANKS)')
print('='*80)

# --- Method 3: Simple Rank Average ---
simple_avg_oof_ranked = np.mean(meta_train_ranked, axis=1)
cv_simple_ranked = roc_auc_score(train[TARGET], simple_avg_oof_ranked)

# Calculate test predictions for all 3 rank-based methods
test_preds_simple_ranked = np.mean(meta_test_ranked, axis=1)

test_preds_optimized_ranked = (optimized_weights_ranked[0] * meta_test_ranked[:, 0] + 
                               optimized_weights_ranked[1] * meta_test_ranked[:, 1] + 
                               optimized_weights_ranked[2] * meta_test_ranked[:, 2] +
                               optimized_weights_ranked[3] * meta_test_ranked[:, 3] +
                               optimized_weights_ranked[4] * meta_test_ranked[:, 4])
            
# Find the best method
best_blend_score = cv_simple_ranked
best_blend_test = test_preds_simple_ranked
blend_name = 'Simple Rank Average'

if cv_l2_lgbm > best_blend_score:
    best_blend_score = cv_l2_lgbm
    best_blend_test = test_preds_l2_lgbm
    blend_name = 'L2 LGBM Stacker (on Ranks)'

if cv_optimized_ranked > best_blend_score:
    best_blend_score = cv_optimized_ranked
    best_blend_test = test_preds_optimized_ranked
    blend_name = 'Optimized Rank Average'

print(f"\n--- Base Models (Multi-Seed CV) ---")
print(f'  LightGBM: {cv_lgb_final:.5f}')
print(f'  CatBoost: {cv_cat_final:.5f}')
print(f'  XGBoost: {cv_xgb_final:.5f}')
print(f'  MLPClassifier: {cv_mlp_final:.5f}')
print(f'  Logistic Regression: {cv_lr_final:.5f}')
print(f'\n--- Ensemble Methods (on Ranks) ---')
print(f'  Simple Rank Average: {cv_simple_ranked:.5f}')
print(f'  Optimized Rank Average: {cv_optimized_ranked:.5f}')
print(f'  L2 LGBM Stacker: {cv_l2_lgbm:.5f}')
print(f'\n--- Best Method ---')
print(f'  {blend_name}: {best_blend_score:.5f}')

# ============================================================================
# SAVE
# ============================================================================
submission['id'] = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')['id']
submission[TARGET] = best_blend_test
submission.to_csv('submission.csv', index=False)

oof_df = pd.DataFrame({'id': train['id'], TARGET: best_blend_score})
oof_df.to_csv(f'oof_predictions_{best_blend_score:.5f}.csv', index=False)

print(f'\n{"="*80}')
print('✅ SAVED: submission.csv')
print(f'✅ SAVED: oof_predictions_{best_blend_score:.5f}.csv')
print(f"Runtime: {(time.time() - start_time)/60:.1f} minutes")
print(f'\n{"="*80}\nFINAL STABLE CV SCORE: {best_blend_score:.5f}')
print('='*80)